# The hypothesis

We assume that pre-trained word embeddings encode grammatical and semantical information. In this work we want to establish whether this type of information is encoded throughout all dimensions of an embedding or only in a handful of dimensions.

We want to assess the following models:
- FlauBERT (small, base-cased, base-uncased, large)
- CamemBERT base
- mBERT base, mBERT large
- XLM base, XLM large

We want to assess the following information:
- Gender information (for nouns and adjectives)
- Grammatical number information (for nouns and adjectives)
- PoS information (nouns, verbs, adjectives, adverbs)
- Semantical class (for nouns)

As the result of the experiment, we want to better unserstand the way these grammatic and semantic features are encoded.

# Methodology

1. For each feature-type we construct a binary vector: 1 if for the word possessing the property (e.g. feminine gender) and 0 for not possessing the property.
2. For each model split the embeddings into train (80% of the embeddings) and test.
3. We can assume that the distribution in the feature vector and in the dimensions are independent - WE don't encode the information at all.
    1. Firstly, on the train set we perform ANOVA test with p-value < 0.01 and find dimensions that are likely to be dependent with the feature vector.
    2. Secondly, on the train set, for each dimension we compute MI with the feature vector. If MI == 0, we can consider the dimension to be indepent from the feature vector (not encoding the feature information); otherwise, we can't consider the dimensions independent.
    3. For both ANOVA and MI, we get a list of dimensions. We also obtain an intersection of such dimensions: we'll call it `Non-independent dimensions`
3. Then we can assume that certain dimensions encode the information better than the others, for this, we can attempt to test for feature importance. For each of the subsequent tests we set a threshold $\alpha$ which will mark how many percents of the test results are considered to be important. We will test several $\alpha$ values
    1. The first test for importance: train a logistic regression classifier for the 2 classes of the feature vector on the train set. Interpret its weights as the importance measure: the higher the absolute weight associated with the dimension is, the higher is the importance of the dimension for encoding the feature.
    2. Repeat a similar experiment but for Perceptron. Top $\alpha$ dimensions with the highest absolute weights are considered to be the most improtant.
    3. For each dimension, compute correlation with the feature vector on the train set. Top $\alpha$ dimensions with the highest correlation to the feature vector are considered important.
    4. As the result, we have 3 more lists of dimensions that should be narrow than the lists from the step #2. We can also find an intersection of all 3 lists and call it `Important dimensions`.
4. For each model, split the train set into 2 parts: where the feature vector == 0 and where it == 1. For each of the subparts compute the median vector, using different dimensions groups:
    1. ANOVA dims
    2. MI dims
    3. Non-independent dims
    4. LR dims (with the threshold $\alpha$)
    5. Perceptron dims (with the threshold $\alpha$)
    6. Correlation dims (with the threshold $\alpha$)
    7. Important dims
    8. All dims
5. For each model, for each dimension group, compute a predicted label for a WE on the test set using MAE from the WE to the median of sublass 0 and subclass 1. Assign the label of the smallest MAE.
6. Compute accuracy for each model for each dimension group and compare. Find the best set of dimensions.
8. Repeat the experiment 5 times and keep only the stable (repeated) dimensions.
